# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
!pip install gmaps

Processing /Users/tess/Library/Caches/pip/wheels/30/bd/87/d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4/gmaps-0.9.0-py2.py3-none-any.whl
  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_data = pd.read_csv('../output_data/cities.csv')
city_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
locations = city_data[['Lat','Lng']]
weight = city_data['Humidity']
gmap = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False)

gmap.add_layer(heat_layer)
gmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
fitting_weather = city_data.loc[(city_data['Max Temp'] < 80) & (city_data['Max Temp'] > 65) 
              & (city_data['Wind Speed'] < 10) 
              & (city_data['Cloudiness'] < 20)].dropna()
fitting_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
211,211,labuhan,18,ID,1558378794,86,-2.54,115.51,75.48,2.80
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = fitting_weather[["City", "Country", "Lat", "Lng"]]
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-24-4f7c3c82a3d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Hotel Name
9,pushkinskiye gory,RU,57.02,28.91,
70,ponta do sol,BR,-20.63,-46.00,
87,ambilobe,MG,-13.19,49.05,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
168,straumen,NO,63.87,11.30,
176,nador,MA,35.17,-2.93,
211,labuhan,ID,-2.54,115.51,
214,mount isa,AU,-20.73,139.49,


In [31]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
parameters = {
    'radius': 5000,
    'types': 'hotel',
    'key': g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    parameters['location'] = f'{lat}, {lng}'
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response_json = requests.get(base_url, params=parameters).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df

/Users/tess/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
9,pushkinskiye gory,RU,57.02,28.91,Pushkinskiye Gory
70,ponta do sol,BR,-20.63,-46.00,Capitólio
87,ambilobe,MG,-13.19,49.05,Ambilobe
88,cidreira,BR,-30.17,-50.22,CIDREIRA
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,Jagalbet
168,straumen,NO,63.87,11.30,Straumen
176,nador,MA,35.17,-2.93,Nador
211,labuhan,ID,-2.54,115.51,SD Negeri Barabai Utara
214,mount isa,AU,-20.73,139.49,Mount Isa


In [ ]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
